In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw


100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




4.4%

100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
type(training_data)

torchvision.datasets.mnist.FashionMNIST

In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0336,  0.0354, -0.0280,  ..., -0.0070, -0.0212,  0.0036],
         [-0.0117, -0.0322, -0.0278,  ..., -0.0244, -0.0205, -0.0248],
         [-0.0155,  0.0242, -0.0348,  ...,  0.0068, -0.0276,  0.0309],
         ...,
         [ 0.0313, -0.0177, -0.0103,  ..., -0.0319,  0.0082,  0.0028],
         [-0.0004, -0.0353,  0.0321,  ..., -0.0286,  0.0242, -0.0337],
         [-0.0026,  0.0252, -0.0355,  ..., -0.0043,  0.0327, -0.0234]],
        requires_grad=True),
 Parameter containing:
 tensor([-1.1965e-02,  1.2689e-02,  2.5080e-02, -2.2930e-02, -3.2450e-02,
         -6.7337e-03, -2.6454e-02, -2.3472e-02, -7.0700e-03,  6.5386e-03,
          1.8774e-03, -1.0819e-02,  6.2408e-03, -2.3547e-02, -9.9533e-03,
          1.0671e-02,  1.5058e-02, -9.4910e-04, -4.7000e-03, -3.0892e-02,
          8.1920e-03, -2.8921e-03,  2.4601e-02,  1.9588e-02, -3.0988e-02,
          5.8759e-03,  2.5979e-02,  3.4220e-02, -3.1650e-02, -1.8682e-02,
          4.1246e-03,  1.2818e-02, -1.34

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305227  [   64/60000]
loss: 2.285669  [ 6464/60000]
loss: 2.262737  [12864/60000]
loss: 2.259619  [19264/60000]
loss: 2.247397  [25664/60000]
loss: 2.204759  [32064/60000]
loss: 2.223742  [38464/60000]
loss: 2.183175  [44864/60000]
loss: 2.178366  [51264/60000]
loss: 2.139631  [57664/60000]
Test Error: 
 Accuracy: 38.7%, Avg loss: 2.142046 

Epoch 2
-------------------------------
loss: 2.159042  [   64/60000]
loss: 2.137180  [ 6464/60000]
loss: 2.086256  [12864/60000]
loss: 2.100464  [19264/60000]
loss: 2.051581  [25664/60000]
loss: 1.981323  [32064/60000]
loss: 2.012892  [38464/60000]
loss: 1.933660  [44864/60000]
loss: 1.939611  [51264/60000]
loss: 1.844631  [57664/60000]
Test Error: 
 Accuracy: 57.4%, Avg loss: 1.866238 

Epoch 3
-------------------------------
loss: 1.907612  [   64/60000]
loss: 1.857291  [ 6464/60000]
loss: 1.762408  [12864/60000]
loss: 1.794612  [19264/60000]
loss: 1.689844  [25664/60000]
loss: 1.640735  [32064/600

In [14]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
